<a href="https://colab.research.google.com/github/casjunior93/Desafio-Dio---Treinamento-da-Rede-YOLO-com-transfer-learning/blob/main/Detec%C3%A7%C3%A3o_com_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio Dio - Treinamento da Rede YOLO com transfer learning

Conectando com o Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!sudo apt-get install tree

In [4]:
!tree /content/gdrive/MyDrive/darknet

/content/gdrive/MyDrive/darknet

0 directories, 0 files


Checando a versão do CUDA - Já vem pré-instalado no Colab

In [5]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Necessário instalar o cuDNN de acordo com a versão do CUDA

Para isso, é preciso [baixar o cuDNN](https://developer.nvidia.com/rdp/cudnn-archive) do site da Nvidia levando em consideração a versão do CUDA verificado anteriormente.

In [7]:
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-linux-x86_64-8.8.1.3_cuda11-archive.tar.xz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

tar (child): gdrive/My Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v11.8.89.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
chmod: cannot access '/usr/local/cuda/include/cudnn.h': No such file or directory
